# **Data importing**

In [ ]:
## Block 1: Download data base

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2024-07-12 21:55:23--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.201.207, 74.125.202.207, 74.125.69.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.201.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  71.2MB/s    in 0.9s    

2024-07-12 21:55:24 (71.2 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
## Block 2: extract data to the directory
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
## Block 3: all of requirment imports
import numpy as np
import random
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os
from os import listdir
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Conv1D,Conv2D, AveragePooling2D,SeparableConv2D, MaxPooling2D
from tensorflow.keras.layers import Lambda,DepthwiseConv2D,Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout , Multiply, Concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv1D, Add, Reshape
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
import numpy as np

In [ ]:
### Block 4: load Validation and train data of Cat and Dog

def data():
  q=[]
  folder_dir = "/tmp/cats_and_dogs_filtered/validation/cats/"
  for images in os.listdir(folder_dir):
    img_path = "/tmp/cats_and_dogs_filtered/validation/cats/"
    img = load_img(img_path+ images, target_size=(150, 150))  # this is a PIL image
    x = img_to_array(img)
    x /= 255
    q.append(x)
  x_valid1 = np.array(q)
  # print(type(x_valid1))
  # print(x_valid1.shape)
  y_valid1 = np.zeros((500,1))
  # print(y_valid1.shape)
  # print("------------------------")
  ### dog train
  qq=[]
  # get the path/directory
  folder_dir = "/tmp/cats_and_dogs_filtered/validation/dogs/"
  for images in os.listdir(folder_dir):
    img_path = "/tmp/cats_and_dogs_filtered/validation/dogs/"
    # print(images)
    # print(img_path+ images)
    img = load_img(img_path+ images, target_size=(150, 150))  # this is a PIL image
    x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
    # print(type(x))
    # print(x.shape)
    qq.append(x)
  x_valid2 = np.array(qq)
  # print(type(x_valid2))
  # print(x_valid2.shape)
  y_valid2 = np.ones((500,1))
  # print(y_valid2.shape)
  x_valid = np.concatenate((x_valid1,x_valid2),axis=0)
  y_valid = np.concatenate((y_valid1,y_valid2),axis=0)
  x_valid , y_valid = shuffle(x_valid , y_valid , random_state=0)
  # print(x_valid.shape)
  # print(y_valid.shape)
  ##########
  # print("====****************===============")
  # load data
  q=[]
  # get the path/directory
  folder_dir = "/tmp/cats_and_dogs_filtered/train/cats/"
  for images in os.listdir(folder_dir):
    img_path = "/tmp/cats_and_dogs_filtered/train/cats/"
    img = load_img(img_path+ images, target_size=(150, 150))  # this is a PIL image
    x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
    x /= 255
    q.append(x)
  x_train1 = np.array(q)
  # print(type(x_train1))
  # print(x_train1.shape)
  y_train1 = np.zeros((1000,1))
  # print(y_train1.shape)
  # print("------------------------")
  ### dog train
  qq=[]
  # get the path/directory
  folder_dir = "/tmp/cats_and_dogs_filtered/train/dogs/"
  for images in os.listdir(folder_dir):
    img_path = "/tmp/cats_and_dogs_filtered/train/dogs/"
    img = load_img(img_path+ images, target_size=(150, 150))  # this is a PIL image
    x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
    x /= 255
    # print(type(x))
    # print(images)
    # print(x)
    # print(x.shape)
    # Rescale by 1/255
    qq.append(x)
  x_train2 = np.array(qq)
  # print(type(x_train2))
  # print(x_train2.shape)
  y_train2 = np.ones((1000,1))
  # print(y_train2.shape)
  ###
  x_train = np.concatenate((x_train1,x_train2),axis=0)
  y_train = np.concatenate((y_train1,y_train2),axis=0)
  ###
  x_train , y_train = shuffle(x_train , y_train , random_state=0)
  print(x_train.shape)
  print(y_train.shape)
  ###
  X_train = x_train
  Y_train = y_train
  X_test  = x_valid
  Y_test  = y_valid
  return X_train , Y_train , X_test , Y_test

In [ ]:
x_train , y_train , x_valid , y_valid = data()

# **Model definition**

In [ ]:
### Block 5: Define the best and desired model

def getmodel():
    img_input = Input(shape=(150, 150, 3))
    # First convolution extracts 16 filters that are 3x3
    # Convolution is followed by max-pooling layer with a 2x2 window
    # x = Conv2D(16, 3, activation='relu')(img_input)
    x = Conv2D(16, 3 )(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling2D(2)(x)
    # Second convolution extracts 32 filters that are 3x3
    # Convolution is followed by max-pooling layer with a 2x2 window
    # x = Conv2D(32, 3, activation='relu')(x)
    x = Conv2D(32, 3 )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling2D(2)(x)
    # Third convolution extracts 64 filters that are 3x3
    # Convolution is followed by max-pooling layer with a 2x2 window
    # x = Conv2D(64, 3, activation='relu')(x)
    x = Conv2D(64, 3 )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling2D(2)(x)
    # Flatten feature map to a 1-dim tensor so we can add fully connected layers
    x = Flatten()(x)
    # Create a fully connected layer with ReLU activation and 512 hidden units
    x = Dense(512, activation='relu')(x)
    # Create output layer with a single node and sigmoid activation
    output = Dense(1, activation='sigmoid')(x)
    # Create model:
    from keras import optimizers
    model = Model(img_input, output)
    model.compile(loss='binary_crossentropy',
                optimizer= 'Adam' ,
                metrics=['acc'])
    return model

# **Starting HYPERAS**

In [ ]:
## Block 6: installing HYPERAS
!pip install --no-deps hyperas==0.4.1
!pip install --no-deps hyperopt==0.1.2

In [ ]:
!pip install utils

In [ ]:
## Block 7: require imports
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
# import utils
# from keras.utils import np_utils

In [ ]:
## Block 8: definition model(1) for HYPERAS
def model(X_train, Y_train, X_test, Y_test):
    img_input = Input(shape=(150, 150, 3))
    # First convolution extracts 16 filters that are 3x3
    # Convolution is followed by max-pooling layer with a 2x2 window
    p = {{choice([16 , 32, 64])}}
    pp = {{choice([3,6,9,12])}}
    x = Conv2D(p , pp, activation='relu')(img_input)
    x = MaxPooling2D(2)(x)
    # Second convolution extracts 32 filters that are 3x3
    # Convolution is followed by max-pooling layer with a 2x2 window
    q = {{choice([16, 32, 64])}}
    qq = {{choice([3,6,9,12])}}
    x = Conv2D(q, qq, activation='relu')(x)
    x = MaxPooling2D(2)(x)
    # Third convolution extracts 64 filters that are 3x3
    # Convolution is followed by max-pooling layer with a 2x2 window
    d = {{choice([8, 16, 32, 64])}}
    dd = {{choice([3,6,9,12])}}
    x = Conv2D(d, dd, activation='relu')(x)
    x = MaxPooling2D(2)(x)
    # Flatten feature map to a 1-dim tensor so we can add fully connected layers
    x = Flatten()(x)
    # Create a fully connected layer with ReLU activation and 512 hidden units
    x = Dense(512, activation='relu')(x)
    # Create output layer with a single node and sigmoid activation
    output = Dense(1, activation='sigmoid')(x)
    # Create model:
    model = Model(img_input, output)
    model.compile(loss='binary_crossentropy',
                optimizer= {{choice([ 'RMSprop','Adam','Nadam', 'SGD' ])}} ,metrics=['acc'])
    ###
    mmmn = {{choice([20,30,40,50,60])}}
    model.fit(X_train, Y_train,
              batch_size= mmmn ,
              epochs=50 ,
              verbose=2,
              validation_data=(X_test, Y_test),validation_steps=1)
    ###
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [ ]:
## Block 9: saving ipython
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fid = drive.ListFile({'q':"title='NewNetHyperas.ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('NewNetHyperas.ipynb')

In [ ]:
## Block 10: running HYPERAS model(1)
best_run, best_model = optim.minimize(model=model,
                                      data=data, max_evals= 6 ,algo=tpe.suggest,
                                      notebook_name='NewNetHyperas'
                                      ,trials = Trials())

X_train, Y_train, X_test, Y_test  = data()
print("-------------")
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("-------------")
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import os
except:
    pass

try:
    import zipfile
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import random
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from tensorflow.keras.preprocessing.image import img_to_array, load_img
except:
    pass

try:
    import os
except:
    pass

try:
    from os import listdir
except:
    pass

try:
    from sklearn.utils import shuffle
except:
    pass

try:
    from tensorflow.keras.optimizers import RMSprop
except:
    pass

try:
    from sklearn.metrics import accuracy_score
except:
    pass

try:
    from sklearn.metrics import precision_score
except:
    pass

try:
    from sklearn.metrics import recall_